# SQL

**Задача**

Компания купила крупный сервис для чтения книг по подписке. Задача — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных**

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.


## Исследование данных

In [1]:
# Импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine 
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
 db_config['pwd'], 
 db_config['host'], 
 db_config['port'], 
 db_config['db']) 
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# Первые строки таблицы books
query = '''
            SELECT * 
            FROM books
        '''
books = pd.io.sql.read_sql(query, con = engine)
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [3]:
# Первые строки таблицы authors
query = '''
            SELECT * 
            FROM authors
        '''
authors = pd.io.sql.read_sql(query, con = engine)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# Первые строки таблицы publishers
query = '''
            SELECT * 
            FROM publishers
        '''
publishers = pd.io.sql.read_sql(query, con = engine)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# Первые строки таблицы ratings
query = '''
            SELECT * 
            FROM ratings
        '''
ratings = pd.io.sql.read_sql(query, con = engine)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# Первые строки таблицы reviews
query = '''
            SELECT * 
            FROM reviews
        '''
reviews = pd.io.sql.read_sql(query, con = engine)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


## Количество книг, выпущенных после 1 января 2000 года

In [7]:
# Посчитайте, сколько книг вышло после 1 января 2000 года;
pd.io.sql.read_sql("SELECT count(book_id) FROM books WHERE publication_date >= '2000-01-01 00:00:00'", con = engine) 

,count
0,821


После 1 января 2000 года была выпущена 821 книга. 

## Количество обзоров и средняя оценка каждой книги 

In [8]:
# Для каждой книги посчитайте количество обзоров и среднюю оценку;
pd.io.sql.read_sql("""
    WITH review_count AS (
        SELECT b.book_id, count(rv.review_id) AS review_count FROM books b
        LEFT JOIN reviews rv ON b.book_id = rv.book_id
        GROUP BY b.book_id
    ), average_rating AS (
        SELECT b.book_id, AVG(rt.rating) AS average_rating FROM books b
        LEFT JOIN ratings rt ON b.book_id = rt.book_id
        GROUP BY b.book_id
    )
    SELECT b.title, rc.review_count, ar.average_rating
    FROM books b
    LEFT JOIN review_count rc ON b.book_id = rc.book_id
    LEFT JOIN average_rating ar ON b.book_id = ar.book_id
    """,
    con = engine) 

,title,review_count,average_rating
0,The Body in the Library (Miss Marple #3),2,4.500000
1,Galápagos,2,4.500000
2,A Tree Grows in Brooklyn,5,4.250000
3,Undaunted Courage: The Pioneering First Missio...,2,4.000000
4,The Prophet,4,4.285714
...,...,...,...
995,Alice in Wonderland,4,4.230769
996,A Woman of Substance (Emma Harte Saga #1),2,5.000000
997,Christine,3,3.428571
998,The Magicians' Guild (Black Magician Trilogy #1),2,3.500000


In [9]:
# Проверим, что сумма по столбцу review_count совпадает с длинной таблицы reviews, а также найдем минимальные и максимальные значения средней оценки и кол-ва отзывов

pd.io.sql.read_sql("""
    WITH review_count AS (
        SELECT b.book_id, count(rv.review_id) AS review_count FROM books b
        LEFT JOIN reviews rv ON b.book_id = rv.book_id
        GROUP BY b.book_id
    ), average_rating AS (
        SELECT b.book_id, AVG(rt.rating) AS average_rating FROM books b
        LEFT JOIN ratings rt ON b.book_id = rt.book_id
        GROUP BY b.book_id
    )
    SELECT
        SUM(s.review_count) AS review_sum,
        MIN(s.review_count) AS min_review_count,
        MAX(s.review_count) AS max_review_count,
        MIN(s.average_rating) AS min_rating,
        MAX(s.average_rating) AS max_rating
    FROM (
        SELECT b.title, rc.review_count, ar.average_rating
        FROM books b
        LEFT JOIN review_count rc ON b.book_id = rc.book_id
        LEFT JOIN average_rating ar ON b.book_id = ar.book_id
    ) s
    
    """,
    con = engine) 

,review_sum,min_review_count,max_review_count,min_rating,max_rating
0,2793.0,0,7,1.5,5.0


Для каждой книги написано несколько обзоров: минимальный рейтинг - 1.5, максимальный - 5, минимальное количество отзывов на книгу -о, максимальное -7. 

## Издательство, выпустившее наибольшее число книг толще 50 страниц

In [10]:
# Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
pd.io.sql.read_sql("""
    SELECT p.publisher, count(b.book_id) AS book_count
    FROM publishers p
    LEFT JOIN books b ON p.publisher_id = b.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY count(b.book_id) DESC
    """,
    con = engine) 

,publisher,book_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


Наибольшее число книг выпустило издательство Penguin Books: всего 42 книги толще 50 страниц. 

## Автор с самой высокой средней оценкой книг

In [11]:
# Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
pd.io.sql.read_sql("""
    -- только книги с 50 и более оценками
    WITH books_most_rated AS (
        SELECT b.book_id, b.author_id
        FROM books b
        LEFT JOIN ratings r ON b.book_id = r.book_id
        GROUP BY b.book_id
        HAVING count(r.rating_id) >= 50
    )
    SELECT a.author, AVG(r.rating) as average_rating
    FROM authors a
    INNER JOIN books_most_rated b ON a.author_id = b.author_id
    LEFT JOIN ratings r ON b.book_id = r.book_id
    GROUP BY a.author_id
    ORDER BY AVG(r.rating) DESC
    """,
    con = engine) 

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


Самый высокий средний рейтинг книги у писательницы Дианы Гэблдон. При этом наибольшее количество оценок от пользователей получили книги Джоан Роулинг, и их рейтинг только на несколько десятых уступает книгам Гэблдон. 

## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [12]:
# Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
pd.io.sql.read_sql("""
    WITH top_raters AS (
    SELECT COUNT(rv.review_id) AS review_count FROM reviews rv WHERE rv.username IN (
        SELECT DISTINCT rt.username FROM ratings rt GROUP BY rt.username HAVING count(rt.rating) > 50
        ) GROUP BY rv.username
    )
    SELECT AVG(review_count) FROM top_raters
    
    """,
    con = engine) 

,avg
0,24.333333


Среднее количество обзоров от пользователей, которые поставили более 50 оценок, - 24.